In [ ]:
import villas.dataprocessing.readtools as rt
from villas.dataprocessing.timeseries import TimeSeries as ts
import matplotlib.pyplot as plt
import re
import numpy as np
import math
import os
import dpsimpy
import glob
import requests

In [ ]:
sim_name = "DP_WSCC9bus_SGReducedOrderVBR"
final_time = 0.1
time_step = 10e-6
with_fault = True
start_time_fault = 0.2
end_time_fault = 0.3
sg_type = "4"
fault_bus_name = "BUS6"
intertia_scaling_factor = 1.0
log_dir = "logs"
solver = dpsimpy.DirectLinearSolverImpl.SparseLU
sim_name_pf = sim_name + "_PF"
log_level = dpsimpy.LogLevel.trace

if time_step < 10e-6:
    log_down_sampling = floor((10e-6) / time_step)
else:
    log_down_sampling = 1.0

def download_grid_data(name, url):
    with open(name, 'wb') as out_file:
        content = requests.get(url, stream=True).content
        out_file.write(content)

filename = 'WSCC-09_Dyn_Fourth'

url = 'https://raw.githubusercontent.com/dpsim-simulator/cim-grid-data/master/WSCC-09/WSCC-09_Dyn_Fourth/WSCC-09_Dyn_Fourth'
download_grid_data(filename+'_EQ.xml', url+'_EQ.xml')
download_grid_data(filename+'_TP.xml', url+'_TP.xml')
download_grid_data(filename+'_SV.xml', url+'_SV.xml')
download_grid_data(filename+'_DI.xml', url+'_DI.xml')
    
files = glob.glob(filename+'_*.xml')

dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name_pf)
reader = dpsimpy.CIMReader(sim_name_pf, log_level, log_level)

system_pf = reader.loadCIM(60, files, dpsimpy.Domain.SP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.PVNode)
system_pf.component('GEN1').modify_power_flow_bus_type(dpsimpy.PowerflowBusType.VD)
logger_pf = dpsimpy.Logger(sim_name_pf)
for node in system_pf.nodes:
    logger_pf.log_attribute(node.name() + ".V", node.attr("v"))
    
sim_pf = dpsimpy.Simulation(sim_name_pf, log_level)
sim_pf.set_system(system_pf)
sim_pf.set_time_step(final_time)
sim_pf.set_final_time(2*final_time)
sim_pf.set_domain(dpsimpy.Domain.SP)
sim_pf.set_solver(dpsimpy.Solver.NRP)
sim_pf.set_solver_component_behaviour(dpsimpy.SolverBehaviour.Initialization)
sim_pf.do_init_from_nodes_and_terminals(True)
sim_pf.add_logger(logger_pf)
sim_pf.run()

In [ ]:
## Dynamic Simulation
def run_simulation(implementation):
    dpsimpy.Logger.set_log_dir(log_dir+"/"+sim_name)
    reader2 = dpsimpy.CIMReader(sim_name, log_level, log_level)
    if sg_type == "3":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG3OrderVBR)
    elif sg_type == "4":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG4OrderVBR)
    elif sg_type == "6b":
        sys = reader2.loadCIM(60, files, dpsimpy.Domain.DP, dpsimpy.PhaseType.Single, dpsimpy.GeneratorType.SG6bOrderVBR)
    else:
        raise Exception("Unsupported reduced-order SG type!")

    fault_dp = dpsimpy.sp.ph1.Switch("Fault", log_level)

    if with_fault == True:
        n1_dp = dpsimpy.dp.SimNode(fault_bus_name, dpsimpy.PhaseType.Single)
        fault_dp.set_parameters(1e12, 0.02*529)
        fault_dp.connect([dpsimpy.dp.SimNode.gnd, n1_dp])
        fault_dp.open()
        sys.add(fault_dp)

    sys.init_with_powerflow(system_pf)
    for comp in sys.components:
       # I assume this is how you access the generators
       if "GEN" in comp.name():
           comp.scale_inertia_constant(intertia_scaling_factor)
           comp.set_model_as_norton_source(False)

    logger = dpsimpy.Logger(sim_name)#, True, log_down_sampling)
    for node in sys.nodes:
        logger.log_attribute(node.name() + ".V", node.attr("v"))

    for comp in sys.components:
        if "GEN" in comp.name():
            logger.log_attribute(comp.name() + ".Tm", comp.attr("Tm"))
            logger.log_attribute(comp.name() + ".Te", comp.attr("Te"))
            logger.log_attribute(comp.name() + ".omega", comp.attr("w_r"))
            logger.log_attribute(comp.name() + ".delta", comp.attr("delta"))

    sim = dpsimpy.Simulation(sim_name, log_level)
    sim.set_system(sys)
    sim.set_domain(dpsimpy.Domain.DP)
    sim.set_solver(dpsimpy.Solver.MNA)
    sim.set_time_step(time_step)
    sim.set_final_time(final_time)
    sim.do_system_matrix_recomputation(True)
    sim.set_direct_solver_implementation(implementation)
    sim.add_logger(logger)

    if with_fault:
        fault_event1 = dpsimpy.event.SwitchEvent(start_time_fault, fault_dp, True)
        fault_event2 = dpsimpy.event.SwitchEvent(end_time_fault, fault_dp, False)
        sim.add_event(fault_event1)
        sim.add_event(fault_event2)

    sim.run()

In [ ]:
%%capture
run_simulation(dpsimpy.DirectLinearSolverImpl.SparseLU)
ts_dpsim_lv_sparse = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_sparse = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
%%capture
run_simulation(dpsimpy.DirectLinearSolverImpl.KLU)
ts_dpsim_lv_klu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_klu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
%%capture
run_simulation(dpsimpy.DirectLinearSolverImpl.DenseLU)
ts_dpsim_lv_denselu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_LeftVector.csv")
ts_dpsim_rv_denselu = rt.read_timeseries_dpsim(dpsimpy.Logger.get_log_dir()+"/"+sim_name+"_RightVector.csv")

In [ ]:
relative_solver_error_tolerance = 4.31e-9
close_to_zero_tolerance = 1e-12

maxerror = 0.0
maxerrorn = 0.0

for entry in ts_dpsim_rv_sparse:
    sparse_values = ts_dpsim_rv_sparse[entry].values
    klu_values = ts_dpsim_rv_klu[entry].values
    denselu_values = ts_dpsim_rv_denselu[entry].values
    
    N = len(klu_values)
    
    relativ_diff_sparse_klu = np.zeros(N, dtype=complex)
    relativ_diff_sparse_denselu = np.zeros(N, dtype=complex)
    
    for i in range(0, N-1):
        if abs(klu_values[i]) > close_to_zero_tolerance:
            relativ_diff_sparse_klu[i] = (sparse_values[i]-klu_values[i])/klu_values[i]
            relativ_diff_sparse_denselu[i] = (sparse_values[i]-denselu_values[i])/klu_values[i]
            if relativ_diff_sparse_klu[i] > maxerror:
                maxerror = relativ_diff_sparse_klu[i]
            if relativ_diff_sparse_denselu[i] > maxerror:
                maxerror = relativ_diff_sparse_denselu[i]
    
    error_1 = np.linalg.norm(relativ_diff_sparse_klu, ord=2)
    error_2 = np.linalg.norm(relativ_diff_sparse_denselu, ord=2)
    print('RV Entry {} Maximum Difference of SparseLU vs. KLU: {}'.format(entry,error_1))
    print('RV Entry {} Maximum Difference of DenseLU vs. SparseLU: {}'.format(entry,error_2))
    if error_1 > maxerrorn:
        maxerrorn = error_1
    if error_2 > maxerrorn:
        maxerrorn = error_2

for entry in ts_dpsim_lv_sparse:
    sparse_values = ts_dpsim_lv_sparse[entry].values
    klu_values = ts_dpsim_lv_klu[entry].values
    denselu_values = ts_dpsim_lv_denselu[entry].values
    
    N = len(klu_values)
    
    relativ_diff_sparse_klu = np.zeros(N, dtype=complex)
    relativ_diff_sparse_denselu = np.zeros(N, dtype=complex)
    
    for i in range(0, N-1):
        if abs(klu_values[i]) > close_to_zero_tolerance:
            relativ_diff_sparse_klu[i] = (sparse_values[i]-klu_values[i])/klu_values[i]
            relativ_diff_sparse_denselu[i] = (sparse_values[i]-denselu_values[i])/klu_values[i]
            if relativ_diff_sparse_klu[i] > maxerror:
                maxerror = relativ_diff_sparse_klu[i]
            if relativ_diff_sparse_denselu[i] > maxerror:
                maxerror = relativ_diff_sparse_denselu[i]
    
    error_1 = np.linalg.norm(relativ_diff_sparse_klu, ord=2)
    error_2 = np.linalg.norm(relativ_diff_sparse_denselu, ord=2)
    print('LV Entry {} Maximum Difference of SparseLU vs. KLU: {}'.format(entry,error_1))
    print('LV Entry {} Maximum Difference of DenseLU vs. SparseLU: {}'.format(entry,error_2))
    if error_1 > maxerrorn:
        maxerrorn = error_1
    if error_2 > maxerrorn:
        maxerrorn = error_2
        
print('Maximum error in all values: {}'.format(maxerror))
print('Maximum norm of errors: {}'.format(maxerrorn))

In [ ]:
assert(maxerror < relative_solver_error_tolerance)
assert(maxerrorn < relative_solver_error_tolerance)